In [1]:
import json
import os
import pickle
import random
import sys

from typing import Callable, Dict, List, Optional
import haiku as hk
import ase
import ase.io
import jax
import jax.numpy as jnp
import numpy as np
import optax
import yaml


from model.datasets import becs_eps_datasets
from model.utils import (
    create_directory_with_random_name,
    compute_avg_num_neighbors,
)
from model.data_utils import (
    get_atomic_number_table_from_zs,
    compute_average_E0s,
)
from model.predictors import predict_becs_eps
from model.optimizer import optimizer
from model.becs_eps_train import BECS_EPS_train
from model.loss import BecsEpsLoss


from model.becs_eps_model import BECS_EPS_model

jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)
np.set_printoptions(precision=3, suppress=True)

In [2]:
with open('data/train_eps.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

save_dir_name = create_directory_with_random_name(
    os.path.splitext('eps_training')[0]
)

2024-03-12-18:18-eps_training-amazing-caren


In [3]:
# Save config and code
with open(f"{save_dir_name}/config.yaml", "w") as f:
    yaml.dump(config, f)
with open(f"{save_dir_name}/train.py", "w") as f:
    with open(sys.argv[0]) as g:
        f.write(g.read())
        
train_loader, valid_loader,test_loader, r_max = becs_eps_datasets(
    r_max = config["cutoff"],
    train_path = config["dataset"]["train_path"],
    #valid_path = config["dataset"]["valid_path"],
    #train_num = config["dataset"]["train_num"],
    valid_num = config["dataset"]["valid_num"],
    n_node = config["dataset"]["num_nodes"],
    n_edge = config["dataset"]["num_edges"],
    n_graph = config["dataset"]["num_graphs"],
)

print(len(train_loader.graphs))
print(len(valid_loader.graphs))


model_fn, params, num_message_passing = BECS_EPS_model(
    r_max=r_max,
    atomic_energies_dict={},
    train_graphs=train_loader.graphs,
    initialize_seed=config["model"]["seed"],
    num_species = config["model"]["num_species"],
    use_sc = True,
    graph_net_steps = config["model"]["num_layers"],
    hidden_irreps = config["model"]["internal_irreps"],
    nonlinearities =  {'e': 'swish', 'o': 'tanh'},
    save_dir_name = save_dir_name,
    reload = config["initialization"]['reload'] if 'reload' in config["initialization"] else None,
)
    
print("num_params:", sum(p.size for p in jax.tree_util.tree_leaves(params)))
    
predictor = jax.jit(
    lambda w, g: predict_becs_eps(lambda *x: model_fn(w, *x), g)
)
    
gradient_transform, steps_per_interval, max_num_intervals = optimizer(
    lr = config["training"]["learning_rate"],
    max_num_intervals = config["training"]["max_num_intervals"],
    steps_per_interval = config["training"]["steps_per_interval"],
    # weight_decay = config["training"]["weight_decay"],
)
optimizer_state = gradient_transform.init(params)
print("optimizer num_params:", sum(p.size for p in jax.tree_util.tree_leaves(optimizer_state)))
    
loss_fn = BecsEpsLoss(
    becs_weight = config["training"]["becs_weight"],
    becs_sum_weight = config["training"]["becs_sum_weight"],
    eps_weight = config["training"]["eps_weight"],
)

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 7507.52it/s]


1420
100
z_table= AtomicNumberTable: (1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83)
Compute the average number of neighbors: 28.767
Do not normalize the radial basis (avg_r_min=None)
Create BECS/EPS (NequIP-based) model with parameters {'use_sc': True, 'graph_net_steps': 3, 'hidden_irreps': '48x0e + 48x0o + 32x1o + 32x1e +24x2o + 24x2e', 'nonlinearities': {'e': 'swish', 'o': 'tanh'}, 'r_max': 5.0, 'avg_num_neighbors': 28.766618979494517, 'avg_r_min': None, 'num_species': 100, 'radial_basis': <function bessel_basis at 0x7f26cccf2c00>, 'radial_envelope': <function soft_envelope at 0x7f26cccf3c40>}
model: hidden_irreps=48x0e+48x0o+32x1o+32x1e+24x2o+24x2e sh_irreps=1x0e+1x1o+1x2e+1x3o 
num_params: 1426672
optimizer num_params: 4280018


In [4]:
BECS_EPS_train(
    predictor,
    params,
    optimizer_state,
    train_loader,
    valid_loader,
    #test_loader,
    gradient_transform,
    loss_fn =loss_fn,
    max_num_intervals = max_num_intervals,
    steps_per_interval = steps_per_interval,
    save_dir_name = save_dir_name,
    patience = config["training"]["patience"],
    #ema_decay = config["training"]["ema_decay"],
)
print('training done!')
    

Started training


eval_train:   0%|                                        | 0/79 [00:00<?, ?it/s]

Compiled function `model` for args:
- n_node=[ 3  8  8  9 10  8  2  8 10 12 10 10 12 12  4 10 12  2 10  2  8 22  0  0] total=192
- n_edge=[112 140 160 166 336 208  32 284 152 152 404 124 352 208  76 156 336  36
 316  52 160  38   0   0] total=4000
cache size: 1


eval_train:   3%|▊                               | 2/79 [00:07<05:03,  3.94s/it]

Compiled function `model` for args:
- n_node=[12  8  3  5  8  3  6  3  5  5  4 12 16 12  7 12  7  0  0  0  0  0  0  0] total=128
- n_edge=[160 464 128 250 400  64 108  64 142 250  56 216 752 288  96 204 358   0
   0   0   0   0   0   0] total=4000
cache size: 2
Compiled function `model` for args:
- n_node=[12 10  8  9 10  8  5  5  9 10  4  6 10  8  9  5] total=128
- n_edge=[760 192 348 280 254 348 116 106 220 360 150  72 148 204 174 268] total=4000
cache size: 3


...

Interval 29: eval_valid: mae_becs=4.0848, 
Interval 29: Time per interval: 16.2s, among which 0.6s for evaluation.


eval_train:  99%|█████████████████████████████▌| 78/79 [00:00<00:00, 144.75it/s]


Interval 30: eval_train: mae_becs=4.1604, 


eval_valid: 100%|████████████████████████████████| 5/5 [00:00<00:00, 139.06it/s]


Interval 30: eval_valid: mae_becs=4.1131, 
Interval 30: Time per interval: 15.5s, among which 0.6s for evaluation.
Training complete
training done!


In [5]:
model_fn, params, num_message_passing = BECS_EPS_model(
    r_max=r_max,
    atomic_energies_dict={},
    train_graphs=[],
    initialize_seed=config["model"]["seed"],
    num_species = config["model"]["num_species"],
    use_sc = True,
    graph_net_steps = config["model"]["num_layers"],
    hidden_irreps = config["model"]["internal_irreps"],
    nonlinearities =  {'e': 'swish', 'o': 'tanh'},
    save_dir_name = save_dir_name,
    reload = '2024-03-12-18:18-eps_training-amazing-caren',
)


Create BECS/EPS (NequIP-based) model with parameters {'use_sc': True, 'graph_net_steps': 3, 'hidden_irreps': '48x0e + 48x0o + 32x1o + 32x1e +24x2o + 24x2e', 'nonlinearities': {'e': 'swish', 'o': 'tanh'}, 'r_max': 5.0, 'avg_num_neighbors': 28.766618979494517, 'avg_r_min': None, 'num_species': 100, 'radial_basis': <function bessel_basis at 0x7f26cccf2c00>, 'radial_envelope': <function soft_envelope at 0x7f26cccf3c40>}


In [6]:
print(len(valid_loader.graphs))

idx = 7

valid_graph = valid_loader.graphs[idx]
# ground truth Born effective charge
print(valid_graph.nodes.becs)

# predicted Born effective charges
pred_becs = predictor(params, valid_graph)
print(pred_becs['becs'])


100
[[[ 1.917  0.     0.   ]
  [ 0.     1.917  0.   ]
  [-0.    -0.     1.917]]

 [[ 1.917  0.     0.   ]
  [ 0.     1.917  0.   ]
  [-0.    -0.     1.917]]

 [[-3.835  0.     0.   ]
  [ 0.    -3.835  0.   ]
  [ 0.     0.    -3.835]]]
[[[-1.956  0.     0.   ]
  [ 0.    -1.956 -0.   ]
  [ 0.    -0.    -1.956]]

 [[-1.956  0.     0.   ]
  [ 0.    -1.956 -0.   ]
  [ 0.    -0.    -1.956]]

 [[ 3.913 -0.    -0.   ]
  [-0.     3.913  0.   ]
  [-0.     0.     3.913]]]


In [7]:


print(valid_graph.globals.eps)

print(pred_becs['eps'])

[[[14.029 -0.    -0.   ]
  [-0.    14.029 -0.   ]
  [-0.    -0.    14.029]]]
[[[11.607 -0.    -0.   ]
  [-0.    11.607  0.   ]
  [-0.     0.    11.607]]]
